In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.stats import spearmanr
from sklearn.utils import resample

cur_dir = "C:/Users/Shavius/Documents/Uni/Year 4/Project/ELLMRPCTFVIS/dev"
data_dir = os.path.join(cur_dir, "data")
    
    

In [ ]:
# Calculate Inter Rater Agreement

def calculate_human_krippendorff

In [ ]:

def calculate_metric_correlations(human_scores, candidate_base, candidate_refined, n_bootsraps=1000, significance=0.05, rounding=3):
    
    if len(human_scores) == 0 or len(candidate_base) == 0 or len(candidate_refined) == 0:
        raise ValueError("Input series must not be empty.")
    
    if len(human_scores) != len(candidate_base) or len(human_scores) != len(candidate_refined):
        raise ValueError(f"Input series must have the same length.{len(human_scores)} != {len(candidate_base)}, {len(human_scores)} != {len(candidate_refined)}")
    
    base_corr, _ = spearmanr(human_scores, candidate_base)
    refined_corr, _ = spearmanr(human_scores, candidate_refined)
    
    n_obervations = len(human_scores)
    data_pairs = np.column_stack((human_scores, candidate_base, candidate_refined))
    
    bootstrap_base_corrs = []
    bootstrap_refined_corrs = []
    
    for i in range(n_bootsraps):
        sample = resample(data_pairs, n_samples=n_obervations, replace=True)
        
        boot_base_corr, _ = spearmanr(sample[:, 0], sample[:, 1])
        bootstrap_base_corrs.append(boot_base_corr)
        
        boot_refined_corr, _ = spearmanr(sample[:, 0], sample[:, 2])
        bootstrap_refined_corrs.append(boot_refined_corr)
    
    ci_lower_base = np.percentile(bootstrap_base_corrs, 100 * significance / 2)
    ci_upper_base = np.percentile(bootstrap_base_corrs, 100 * (1 - significance / 2))
    
    ci_lower_refined = np.percentile(bootstrap_refined_corrs, 100 * significance / 2)
    ci_upper_refined = np.percentile(bootstrap_refined_corrs, 100 * (1 - significance / 2))
    
    base_corr = round(base_corr, rounding)
    ci_lower_base = round(ci_lower_base, rounding)
    ci_upper_base = round(ci_upper_base, rounding)
    refined_corr = round(refined_corr, rounding)
    ci_lower_refined = round(ci_lower_refined, rounding)
    ci_upper_refined = round(ci_upper_refined, rounding)
    
    return base_corr, (ci_lower_base, ci_upper_base), refined_corr, (ci_lower_refined, ci_upper_refined)
    
def evaluate_model_result(model_name, prefix):
    human_scores = pd.read_csv(os.path.join(data_dir, f"{prefix}_annotations_average.csv"), index_col=0)["consistency"].values
    base_scores = pd.read_csv(os.path.join(data_dir, f"{prefix}_baseline_output_{model_name}.csv"), index_col=0)["consistency"].values
    refined_scores = pd.read_csv(os.path.join(data_dir, f"{prefix}_combined_output_{model_name}.csv"), index_col=0)["consistency"].values
    
    base_corr, (base_lower_ci, base_upper_ci), refined_corr, (refined_lower_ci, refined_upper_ci) = calculate_metric_correlations(human_scores, base_scores, refined_scores, significance=0.1)

    print(f"Base model correlation: {base_corr} ({base_lower_ci}-{base_upper_ci})")
    print(f"Refined model correlation: {refined_corr} ({refined_lower_ci}-{refined_upper_ci})")
    return base_corr, refined_corr, (base_lower_ci, base_upper_ci), (refined_lower_ci, refined_upper_ci)